In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [3]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [4]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [5]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [6]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

### Задача 1*
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [7]:
def calc_logloss(y, y_pred, eps=1e-15):
    # ограничит предсказание значениями 1e-15 и 1 - 1e-15 
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    
    return err

In [8]:
y1 = np.array([1, 0])

In [9]:
# использование нормальных значений
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [10]:
# использование 0 и 1
y_pred2 = np.array([0, 1])
calc_logloss(y1, y_pred2)

34.53917619362578

### Задача 2
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [11]:
# код функции тот же самый, что и в методичке, но подхватывается уже новая версия calc_logloss
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W, err

In [12]:
# Чем больше количество итераций и коэфициент обучения тем медленне работает предсказание и тем ниже значение ошибки
W, err = eval_model(X_st, y, iterations=10000, eta=42)
# Альтернативные параметры для проверки зависимости между переобучением и гиперпараметрами модели
# W, err = eval_model(X_st, y, iterations=100, eta=10)
err

0 [ -3.29715467 -14.6470242    0.37388435  -4.13843283] 1.1785958344356262
10 [ -3.84719604 -15.68265117   6.53343301  10.06564547] 5.407107479646821
20 [ -5.96483274 -18.42578336  10.99529928  21.95453995] 7.828182836418551
30 [-10.15891717 -22.09876988  14.10637274  28.2892994 ] 5.916795846115139
40 [-12.0971529  -20.02766324  15.4578092   36.72554313] 3.4794070637467582
50 [-15.18985471 -13.12036272  10.01576691  33.63284309] 2.181273835303843
60 [-19.48597898  -9.15855557   4.87486494  28.9496718 ] 1.3048680468257763
70 [-22.14898967  -6.18704278  -1.26057625  25.50726982] 0.6695345627233119
80 [-26.65475679 -18.24947128  -3.26040769  22.2274953 ] 6.479051628544935
90 [-28.81876932 -20.0744361    0.6152993   31.61935683] 2.818912522603524


1.529142039707478

### Задача 3
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [13]:
def calc_pred_proba(X, w):
    y_pred_proba = np.squeeze(sigmoid(np.dot(X, w)))
    return y_pred_proba

In [14]:
calc_pred_proba(X_st, W)

array([2.86191533e-02, 1.07499710e-01, 1.00000000e+00, 1.25492717e-21,
       9.99999999e-01, 9.99950534e-01, 1.00000000e+00, 1.47103708e-06,
       2.02991481e-02, 1.00000000e+00])

### Задача 4
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [15]:
def calc_pred(X, w):
    m = X.shape[0]
    
    y_pred = np.zeros(m)

    A = np.squeeze(sigmoid(np.dot(X, w)))

    # За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(A.shape[0]):
        if (A[i] > 0.5): 
            y_pred[i] = 1
        elif (A[i] <= 0.5):
            y_pred[i] = 0
    
    return y_pred

In [16]:
y_pred = calc_pred(X_st, W)
y_pred

array([0., 0., 1., 0., 1., 1., 1., 0., 0., 1.])

### Задача 5*
Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [17]:
def get_accuracy(pred, y, size):
    accuracy = np.sum(y_pred == y) / size
    return accuracy

In [18]:
get_accuracy(y_pred, y, X_st.shape[0])

0.8

In [19]:
cm = confusion_matrix(y, y_pred)
cm

array([[4, 1],
       [1, 4]], dtype=int64)

In [20]:
def get_precisions(c_matrix, class_num):
    class_num -= 1
    precisions = c_matrix[class_num][class_num]/np.sum(c_matrix[class_num, :])
    return precisions

In [21]:
get_precisions(cm, 2)

0.8

In [22]:
def get_recall(c_matrix, class_num):
    class_num -= 1
    recall = c_matrix[class_num][class_num]/np.sum(c_matrix[:, class_num])
    return recall

In [23]:
get_recall(cm, 2)

0.8

### Задача 6
Могла ли модель переобучиться? Почему?

Думаю так и произошло из-за черезмерно маленькой выборки и того что до упора выкручены коэффициенты в функции eval_model.
т.е. модель подогналась под конкретные значения.